<a href="https://colab.research.google.com/github/ahatasham5/NewspaperSummarizationLLM/blob/main/Newspaper_Summarize_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers accelerate datasets trl peft langchain langchain-community evaluate

# Import libraries
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate.utils import release_memory
import torch
from datasets import Dataset
from trl import SFTTrainer
from peft import LoraConfig, PeftModel
import pandas as pd
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, HTMLHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import evaluate
import transformers
from langchain.llms.base import LLM
from typing import Any
import warnings
import gc
import random
import numpy as np

# Set seed for reproducibility
SEED = 42

# Set seed for PyTorch
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# Set seed for NumPy
np.random.seed(SEED)

# Set seed for Python random module
random.seed(SEED)

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/TextSummerize/winningModel/

/content/drive/MyDrive/TextSummerize/winningModel


In [ ]:
ls

kaggle_winning_solutions_methods.csv         method_category_mapping.csv
kaggle_winning_solutions_methods_detail.csv


In [ ]:
!pip install --upgrade huggingface_hub

In [ ]:


# Access the CSV file from Drive
writeups = pd.read_csv('kaggle_winning_solutions_methods.csv')
writeups = writeups.drop_duplicates(subset=['link', 'writeup']).reset_index(drop=True)

# Access the Gemma model (after agreeing to license):
model_name = "google/gemma-7b"  # Choose from available Gemma models (e.g., gemma-2b, gemma-7b)

# Add the following code to authenticate with HuggingFace
from huggingface_hub import HfApi

# Create a HuggingFace API object
api = HfApi()

# Get the access token from the HuggingFace API
access_token = api.token

# Set the access token for the Transformers library
transformers.set_access_token(access_token)

# Now load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 5. Proceed with your code using the loaded Gemma model
# (Your code for using the model goes here)
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 5. Proceed with your code using the loaded Gemma model
# (Your code for using the model goes here)


AttributeError: module transformers has no attribute set_access_token